In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [3]:
#prices
dam = pd.read_csv('dam.csv')
fuels = pd.read_csv('fuels.csv')
cap = pd.read_csv('cap.csv')

In [6]:
locs = ['GJ', 'LI', 'NYC']

gen = {}

for loc in locs:
    gen[loc] = pd.read_csv(f'../data\Generating_Capacity_{loc}_11-23.csv')

gen['NYCA'] = pd.read_csv('../data\Generating_Capacity_NYCA_04-23.csv')

,Date,NYCA,NYC,LI,G-J
0,2006-01-31,71.063608,77.892083,90.131169,75.150094
1,2006-02-28,66.316258,72.224866,93.183423,69.285882
2,2006-03-31,61.063118,69.330995,83.718024,63.911855
3,2006-04-30,60.440474,66.244896,87.301433,63.217510
4,2006-05-31,57.136391,62.234140,75.975242,60.800796
